**The purpose of this notebook is to organize the results from all trials, (each stored in its own dictionary), and consolidate them into a numpy array containing the statistics of all trials for a specific set of parameter settings (e.g. gamma = 10^-4 ~ 10^8 @ noise=0.3, two 5-unit hidden layers)**

In [2]:
#system dependent. Add the dictionaries to the path.
from matplotlib import pyplot as plt
from scipy.io import loadmat
import tensorflow as tf
import sys
import time
from pathlib import Path
import math as m
from tqdm import tqdm
import numpy as np

In [2]:
# Set a whole bunch of parameters
spy = 60 * 60 * 24 * 365.25
rhoi = 910.
rhow = 1028.
delta = 1. - rhoi / rhow
g = 9.81
a = 0.3 / spy
Q0 = 4.0e5 / spy
H0 = 1.0e3
B0 = 1.4688e8
n = 3

# Set scalings
Z0 = a ** (1/(n+1)) * (4 * B0) ** (n / (n + 1)) / (rhoi * g * delta) ** (n/(n + 1))
U0 = 400 / spy
Lx = U0 * Z0 / a
h0 = H0 / Z0; q0 = Q0 / (U0 * Z0)
nu_star = (2 * B0) / ( rhoi * g * delta * Z0) * (U0 / Lx) ** (1 / n)
A0 = (a * Lx) / (U0 * Z0)

def analytic_h_constantB(x):
    return ((A0 * h0 ** (n + 1) * (A0 * x + q0) ** (n + 1)) / (
            A0 * q0 ** (n + 1) - (q0 * h0) ** (n + 1) + (h0 * (A0 * x + q0)) ** (n + 1))) ** (1 / (n + 1))

def analytic_u_constantB(x):
    return (A0 * x + q0) / analytic_h_constantB(x)

x_star = np.linspace(start = 0.0, stop = 1.0, num = 401)

def compute_uerr(u_pred):
    return np.mean(np.square(u_pred-analytic_u_constantB(x_star)))

def compute_herr(h_pred):
    return np.mean(np.square(h_pred-analytic_h_constantB(x_star)))

**1) Reload compressed result dictionaries from a set of trials into python dictionaries. We temporarily store results from separate trials into individual dictionaries.**

In [2]:
num_trial_list = [501] #number of trials for each set of results being loaded
path_list = ['n3_log_gamma/r'] # the folder where all result dictionaries are stored, plus the "file prefix" of the results
# e.g. if the result dictionaries are named 1st trial = 'r1', 2nd trial = 'r2', pass in /result_folder/r 

dictstr_list = ['n3r'] #choose the prefix you want to call your loaded results. e.g. with this setting the resulting dictionaries 
#will be called n3r1 (first trial result), n3r2 (second trial result), etc

for i in range(len(path_list)):
    num_trials = num_trial_list[i]
    dictstr = dictstr_list[i]
    path = path_list[i]
    
    for j in tqdm(range(num_trials)):
        exec(f"{dictstr}r{j} = loadmat('{path}{j}')")

100%|███████████████████████████████████████████████████████████████████████████████| 501/501 [00:01<00:00, 500.04it/s]


**2) Combine results from a set of trials into a numpy array. We define two helper functions. 'error_array()' creates numpy arrays of average prediction error for all trials, with separate columns for error in u,h, and B. deviation_array() performs a similar operation but for u-deviation and h-deviation (see paper for definitions.)**

In [ ]:
def error_array(g_index, exp, num_trials): #specify index of gamma-value, and name of the experiment (dictstr)
    u_errs = []
    h_errs = []
    b_errs = []
    
    for i in range(num_trials):
        exec(f"u_errs.append(compute_uerr({exp}r{i}['u_p'][{g_index}]))")
        exec(f"h_errs.append(compute_herr({exp}r{i}['h_p'][{g_index}]))")
        exec(f"b_errs.append({exp}r{i}['berrs'][0][{g_index}])")

    u_errs = np.asarray(u_errs)
    h_errs = np.asarray(h_errs)
    b_errs = np.asarray(b_errs)
    
    return u_errs, h_errs, b_errs

def deviation_array(g_index, exp, num_trials): #specify index of gamma-value, and name of the experiment (dictstr)
    u_devs = []
    h_devs = []
    b_errs = []
    
#return np.mean(np.square(u_pred-analytic_u_constantB(x_star)))

    for i in range(num_trials):
        exec(f"u_devs.append(np.mean(np.square(({exp}r{i}['u_p'][{g_index}]-{exp}r{i}['u_sampled'][{g_index}]))))")
        exec(f"h_devs.append(np.mean(np.square(({exp}r{i}['h_p'][{g_index}]-{exp}r{i}['h_sampled'][{g_index}]))))")
        exec(f"b_errs.append({exp}r{i}['berrs'][0][{g_index}])")

    u_devs = np.asarray(u_devs)
    h_devs = np.asarray(h_devs)
    b_errs = np.asarray(b_errs)
    
    return u_devs, h_devs, b_errs

**Example code for a set of trials with noise = 0.3 and 13 values of gamma. Replace arguments of "error_array" and "deviation_array" with the dictionary prefix of the desired set of trials.**

In [ ]:
trial_num = 501 #number of trials
gamma_num = 13 # number of gamma values tested
set_name = 'n3' #dictionary prefix for desired set of trials

#Create a 13 (gamma values) by 3 (variables) by 501 (trials) numpy array.
n3_errs = np.zeros(shape = (gamma_num,3,trial_num))
for i in range(13):
    u_errs, h_errs, b_errs = error_array(i,set_name,trial_num)
    n2_errs[i][0] = u_errs
    n2_errs[i][1] = h_errs
    n2_errs[i][2] = b_errs
np.save('n3_errs.npy', n3_errs)

n3_devs = np.zeros(shape = (13,3,501))
for i in range(13):
    u_devs, h_devs, b_errs = deviation_array(i,set_name,trial_num)
    n3_devs[i][0] = u_devs
    n3_devs[i][1] = h_devs
    n3_devs[i][2] = b_errs
np.save('n3_devs.npy', n3_devs)